In [ ]:
use_gdrive = True

In [ ]:
import os
import random
import cv2
from tqdm import tqdm
import pandas as pd

if use_gdrive:
  from google.colab import drive
  drive.mount('/content/drive')

data_dir = "/content/drive/MyDrive/CV_Final_Project"
isic_dataset_dir = data_dir + '/isic_dataset'
hair_dir = isic_dataset_dir + '/melanoma_hairs'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
MAX_HAIRS = 20

def AugmentHairs(img):
    """
    Args:
        img (PIL Image): Image to draw hairs on.

    Returns:
        PIL Image: Image with drawn hairs.
    """
    n_hairs = random.randint(0, MAX_HAIRS)
    # print("# of hairs added:", n_hairs)

    if not n_hairs:
        return img

    height, width, _ = img.shape  # target image width and height
    hair_images = [im for im in os.listdir(hair_dir) if 'png' in im]

    for _ in range(n_hairs):
        hair = cv2.imread(os.path.join(hair_dir, random.choice(hair_images)))
        hair = cv2.flip(hair, random.choice([-1, 0, 1]))
        hair = cv2.rotate(hair, random.choice([0, 1, 2]))

        # Randomly resize the hair image
        resize_factor = random.uniform(0.5, 5)
        hair = cv2.resize(hair, (int(hair.shape[1] * resize_factor), int(hair.shape[0] * resize_factor)))

        img_height, img_width, _ = img.shape
        hair_height, hair_width, _ = hair.shape

        while hair_height > img_height or hair_width > img_width:
            hair = cv2.resize(hair, (int(hair.shape[1] * 0.8), int(hair.shape[0] * 0.8)))
            hair_height, hair_width, _ = hair.shape

        roi_ho = random.randint(0, img_height - hair_height)
        roi_wo = random.randint(0, img_width - hair_width)

        roi = img[roi_ho:roi_ho + hair_height, roi_wo:roi_wo + hair_width]

        # Creating a mask and inverse mask
        img2gray = cv2.cvtColor(hair, cv2.COLOR_BGR2GRAY)
        ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
        mask_inv = cv2.bitwise_not(mask)

        # Now black-out the area of hair in ROI
        img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

        # Take only region of hair from hair image.
        hair_fg = cv2.bitwise_and(hair, hair, mask=mask)

        # Put hair in ROI and modify the target image
        dst = cv2.add(img_bg, hair_fg)

        img[roi_ho:roi_ho + hair_height, roi_wo:roi_wo + hair_width] = dst

    return img

##Augment training data

In [ ]:
input_dir = isic_dataset_dir + '/train'
output_dir = isic_dataset_dir + '/train_augmented'

counter = 0
data = pd.read_csv(os.path.join(data_dir, 'training_dataset.csv'))

for idx in range(len(data)):
    # Load the input JPG image
    img_name = data.iloc[idx, data.columns.get_loc("image_name")]
    filename = img_name + ".jpg"
    img_path = os.path.join(input_dir, filename)
    if not os.path.exists(img_path):
        print(f"Image file '{img_path}' not found.")
    img = cv2.imread(img_path)

    # Apply hair augmentation
    augmented_img = AugmentHairs(img)

    # Save the augmented image as a new JPG file
    output_image_path = os.path.join(output_dir, f"augmented_{filename}")
    cv2.imwrite(output_image_path, augmented_img)

    print(f"Augmented image {counter} saved successfully: {output_image_path}")
    counter += 1

Augmented image 0 saved successfully: /content/drive/MyDrive/CV_Final_Project/isic_dataset/train_augmented/augmented_ISIC_4493482.jpg
Augmented image 1 saved successfully: /content/drive/MyDrive/CV_Final_Project/isic_dataset/train_augmented/augmented_ISIC_4868926.jpg
Augmented image 2 saved successfully: /content/drive/MyDrive/CV_Final_Project/isic_dataset/train_augmented/augmented_ISIC_1761138.jpg
Augmented image 3 saved successfully: /content/drive/MyDrive/CV_Final_Project/isic_dataset/train_augmented/augmented_ISIC_6299511.jpg
Augmented image 4 saved successfully: /content/drive/MyDrive/CV_Final_Project/isic_dataset/train_augmented/augmented_ISIC_2641026.jpg
Augmented image 5 saved successfully: /content/drive/MyDrive/CV_Final_Project/isic_dataset/train_augmented/augmented_ISIC_4425657.jpg
Augmented image 6 saved successfully: /content/drive/MyDrive/CV_Final_Project/isic_dataset/train_augmented/augmented_ISIC_5356775.jpg
Augmented image 7 saved successfully: /content/drive/MyDrive/C

##Augment testing data

In [62]:
input_dir = isic_dataset_dir + '/train'
output_dir = isic_dataset_dir + '/test_augmented'

counter = 0
data = pd.read_csv(os.path.join(data_dir, 'testing_dataset.csv'))

for idx in range(len(data)):
    # Load the input JPG image
    img_name = data.iloc[idx, data.columns.get_loc("image_name")]
    filename = img_name + ".jpg"
    img_path = os.path.join(input_dir, filename)
    if not os.path.exists(img_path):
        print(f"Image file '{img_path}' not found.")
    img = cv2.imread(img_path)

    # Apply hair augmentation
    augmented_img = AugmentHairs(img)

    # Save the augmented image as a new JPG file
    output_image_path = os.path.join(output_dir, f"augmented_{filename}")
    cv2.imwrite(output_image_path, augmented_img)

    print(f"Augmented image {counter} saved successfully: {output_image_path}")
    counter += 1

Augmented image 0 saved successfully: /content/drive/MyDrive/CV_Final_Project/isic_dataset/test_augmented/augmented_ISIC_7608687.jpg
Augmented image 1 saved successfully: /content/drive/MyDrive/CV_Final_Project/isic_dataset/test_augmented/augmented_ISIC_6255113.jpg
Augmented image 2 saved successfully: /content/drive/MyDrive/CV_Final_Project/isic_dataset/test_augmented/augmented_ISIC_3782057.jpg
Augmented image 3 saved successfully: /content/drive/MyDrive/CV_Final_Project/isic_dataset/test_augmented/augmented_ISIC_2600276.jpg
Augmented image 4 saved successfully: /content/drive/MyDrive/CV_Final_Project/isic_dataset/test_augmented/augmented_ISIC_2607827.jpg
Augmented image 5 saved successfully: /content/drive/MyDrive/CV_Final_Project/isic_dataset/test_augmented/augmented_ISIC_2072219.jpg
Augmented image 6 saved successfully: /content/drive/MyDrive/CV_Final_Project/isic_dataset/test_augmented/augmented_ISIC_9199429.jpg
Augmented image 7 saved successfully: /content/drive/MyDrive/CV_Final